# 10.1 최근 영업일 기준 데이터 받기
- 네이버 금융에서 날짜를 받아오는 예제
- subtop_sise_graph2인 div 태그 -> 클래스가 subtop_chart_note인 ul태그 -> li 태그 -> 클래스가 tah인 span태그 에 위치해 있다는 걸 알 수 있다.

In [1]:
import requests as rq
from bs4 import BeautifulSoup


# 10-1_ex_01
url = "https://finance.naver.com/sise/sise_deposit.nhn"
data = rq.get(url)
data_html = BeautifulSoup(data.content)
parse_day = data_html.select_one(
    "div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah"
).text

print("paras_data : ", parse_day)



paras_data :    |  2024.02.27


- 정규 표현식으로 날짜를 yyyymmdd 형태로 변경

In [3]:
import re

# 10-1_ex_02
biz_day = re.findall("[0-9]+", parse_day)
biz_day = "".join(biz_day)

print("biz_day : ", biz_day)

biz_day :  20240227


# 10.2 한국거래소의 업종분류 현황 및 개별지표 크롤링
### 업종분류 현황 크롤링
- KRX 정보데이터시스템 에서 [기본통계 -> 주식 -> 세부안내] 부분의 **업종분류 현황**과 **개별종목**을 크롤링 하는 예제
- 다운로드 > CSV를 눌렀을 때 Network의 request와 response 내용을 보고 구현한 예제이다.
- 어떤 데이트를 원하는지 'generate.cmd'의 payload에 적어서 Server에 보냄.
- 응답으로 온 code를 'download.cmd'의 payload와 같이 적어서 Server에 보내면 데이터가 응답으로 옴.
    - 단, 만약 응답은 200으로 성공인데 빈 내용으로 응답이 온다면 'genrat.cmd'로 보낸 내용이 맞는지 확인해보자
- pandas 패키지로 생성된 내용을 커맨드 환경에서 출력하고 싶다면 꼭 **print함수**를 써야 한다.
    -jupyter에서는 print함수 없이 출력 가능

In [12]:
import requests as rq
from io import BytesIO
import pandas as pd

gen_otp_url = "http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd"
gen_otp_stk = {
    "locale": "ko_KR",
    "mktId": "STK",
    "trdDd": biz_day,
    "money": "1",
    "csvxls_isNo": "false",
    "name": "fileDown",
    "url": "dbms/MDC/STAT/standard/MDCSTAT03901",
}

headers = {"Referer": "http://data.krx.co.kr/contents/MDC/MDI/mdiLoader"}
otp_stk = rq.post(gen_otp_url, gen_otp_stk, headers=headers).text

print("otp_stk : ", otp_stk)

down_url = "http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd"
down_sector_stk = rq.post(down_url, {"code": otp_stk}, headers=headers)
# print(down_sector_stk.content)

# print(BytesIO(down_sector_stk.content))
sector_stk = pd.read_csv(BytesIO(down_sector_stk.content), encoding="EUC-KR")

sector_stk.head()

otp_stk :  8mP8X2TPInN/FWOaGOG7ZFqSw3zf+dJyMrYUev458RQRtSksuLS7Bnxpl86F7dAOkunw9BBwugQaSjGAcH15eWpElTqRthT7iBANaNWly3YtBgM+EFJCxYg3zco1gIgRZqIo4cIzoURnTI8+MmkJ4v/rk8yudrOQ53ef0cNipdpCT2QuimcLoNhc1Lfcxcp2zXt/wiFT3lY4zZLV84P+nFYo9mrWvGCVTr7yVbjg2wtc9glyj0xDQ2lZrfrchZWN


,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,095570,AJ네트웍스,KOSPI,서비스업,4815,-50,-1.03,217892034585
1,006840,AK홀딩스,KOSPI,기타금융,16510,-390,-2.31,218717232110
2,027410,BGF,KOSPI,기타금융,3880,-20,-0.51,371381149080
3,282330,BGF리테일,KOSPI,유통업,130000,-1200,-0.91,2246907780000
4,138930,BNK금융지주,KOSPI,기타금융,7720,0,0.00,2486522741360


- 위와 같은 방법으로 코스닥 데이터 받아오는 예제

In [11]:
gen_otp_ksq = {
    "locale": "ko_KR",
    "mktId": "KSQ",
    "trdDd": biz_day,
    "money": "1",
    "csvxls_isNo": "false",
    "name": "fileDown",
    "url": "dbms/MDC/STAT/standard/MDCSTAT03901",
}

otp_ksq = rq.post(gen_otp_url, gen_otp_ksq, headers=headers).text

down_sector_ksq = rq.post(down_url, {"code": otp_ksq}, headers=headers)
sector_ksq = pd.read_csv(BytesIO(down_sector_ksq.content), encoding="EUC-KR")

sector_ksq.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,060310,3S,KOSDAQ,기계·장비,2950,-45,-1.50,143183093900
1,054620,APS,KOSDAQ,금융,6820,-320,-4.48,139088587220
2,265520,AP시스템,KOSDAQ,반도체,20600,-900,-4.19,314797272600
3,211270,AP위성,KOSDAQ,통신장비,15490,-290,-1.84,233624888960
4,139050,BF랩스,KOSDAQ,컴퓨터서비스,7840,-20,-0.25,66990416640


- 코스피 데이터와 코스닥 데이터를 하나로 합침

In [10]:
krx_sector = pd.concat([sector_stk, sector_ksq]).reset_index(drop=True) # 두 데이터 합치기
krx_sector['종목명'] = krx_sector['종목명'].str.strip() # 종목명에 공백 있는 경우 제거
krx_sector['기준일'] = biz_day # 데이터 기준일에 해당하는 [기준일] 열을 추가

krx_sector.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일
0,095570,AJ네트웍스,KOSPI,서비스업,4815,-50,-1.03,217892034585,20240227
1,006840,AK홀딩스,KOSPI,기타금융,16510,-390,-2.31,218717232110,20240227
2,027410,BGF,KOSPI,기타금융,3880,-20,-0.51,371381149080,20240227
3,282330,BGF리테일,KOSPI,유통업,130000,-1200,-0.91,2246907780000,20240227
4,138930,BNK금융지주,KOSPI,기타금융,7720,0,0.00,2486522741360,20240227


### 개별종목 지표 크롤링
- 업종 분류 현황 가져오는 방법과 같음.
- headers에서 추가하는 Referer 속성은 꼭 실제 요청 URL과 같을 필요 없이. 비슷한 모든 요청의 공통 부분만 추가해서 보내도 되는 것 같다.

In [9]:
import requests as rq
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
    'searchType' : '1',
    'mktId' : 'ALL',
    'trdDd' : biz_day,
    'csvxls_isNo' : 'false',
    'name' : 'fileDown',
    'url' : 'dbms/MDC/STAT/standard/MDCSTAT03501'
}
headers = {'Referer' : 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp = rq.post(gen_otp_url, gen_otp_data, headers=headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
krx_ind = rq.post(down_url, {'code' : otp}, headers=headers)

krx_ind = pd.read_csv(BytesIO(krx_ind.content), encoding='EUC-KR')
krx_ind['종목명'] = krx_ind['종목명'].str.strip()
krx_ind['기준일'] = biz_day

krx_ind.head()

,종목코드,종목명,종가,대비,등락률,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률,기준일
0,060310,3S,2950,-45,-1.50,30.0,98.33,NaN,NaN,947.0,3.12,0,0.00,20240227
1,095570,AJ네트웍스,4815,-50,-1.03,201.0,23.96,663.0,7.26,8076.0,0.60,270,5.61,20240227
2,006840,AK홀딩스,16510,-390,-2.31,NaN,NaN,NaN,NaN,41948.0,0.39,200,1.21,20240227
3,054620,APS,6820,-320,-4.48,505.0,13.50,NaN,NaN,10864.0,0.63,0,0.00,20240227
4,265520,AP시스템,20600,-900,-4.19,5463.0,3.77,NaN,NaN,17980.0,1.15,270,1.31,20240227


## 10.2.3 데이터 정리
- 두 데이터에 공통으로 존재하지 않는 종목 출력

In [13]:
diff = list(set(krx_sector['종목명']).symmetric_difference(set(krx_ind['종목명'])))
print(diff)

['미래에셋맵스리츠', '엘브이엠씨홀딩스', '프레스티지바이오파마', '씨엑스아이', '한화리츠', '크리스탈신소재', '디앤디플랫폼리츠', '신한서부티엔디리츠', '맥쿼리인프라', '미래에셋글로벌리츠', '맵스리얼티1', '이리츠코크렙', '코람코라이프인프라리츠', 'ESR켄달스퀘어리츠', '이스트아시아홀딩스', 'GRT', '글로벌에스엠', '한국ANKOR유전', '고스트스튜디오', '신한알파리츠', '윙입푸드', '브랜드엑스코퍼레이션 (락)', '헝셩그룹', 'SBI핀테크솔루션즈', '삼성FN리츠', '케이탑리츠', '하나금융지주 (락)', '오가닉티코스메틱', '코오롱티슈진', 'JTC', '한국패러랠', '골든센츄리', 'SK리츠', '코람코더원리츠', '잉글우드랩', 'NH프라임리츠', '하나금융지주', '컬러레이', '롯데리츠', '엑세스바이오', '에이리츠', '이지스밸류리츠', '이지스레지던스리츠', 'NH올원리츠', 'KB스타리츠', '네오이뮨텍', '제이알글로벌리츠', '마스턴프리미어리츠', '애머릿지', '소마젠', '브랜드엑스코퍼레이션', '로스웰', '모두투어리츠']


- 코스피&코스닥 데이터와 개별종목 지표를 합치는 예제